In [ ]:
using JuMP
#using GLPKMathProgInterface
using CPLEX

# Usar callbacks

Una de las grandes ventajas de JuMP es la facilidad con la que podemos llamar a callbacks. Construiremos dos ejemplos.


## Uncapacitated lot-sizing problem

El modelo es el siguiente:

$\mbox{[MIN]} \sum_{t\in T} (K_t y_t + c_t x_t + h_t s_t )$ 

$s_{t-1}+x_t = s_t+d_t \qquad t\in T$

$0\leq x_t\leq My_t \qquad t\in T$

$x_t,s_t \geq 0\qquad t\in T$

$y_t \in \{0,1\}\qquad t\in T$

Hay un conjunto de desigualdades para este problema que cumplen:

\begin{equation}
\sum_{j\in L\setminus S}x_j + \sum_{j\in S}d_{jl} y_{j} \geq d_{1l},\qquad \forall L\in \{1,...,l\}, S\subseteq L
\end{equation}

(Explicación de la restricción en: http://www.dcc.fc.up.pt/~jpp/WOW/Wolsey-LS2.pdf página 16, podemos mirarlo como lo que se fabrica en unos periodos más lo máximo que se puede fabricar en otros debe cubrir la demanda hasta un periodo). Específicamente, conviene recordar que $d_{jl}=\sum_{t=j}^{l} d_t$.

Ejemplo tomado de: http://sbebo.github.io/blog/blog/2015/06/10/julia/

In [ ]:
function separate(T::Int64, sumd::Array{Float64, 2}, y_val, x_val, y, x)
    TOL=1E-6
    S = zeros(Bool, T)
    for l in 1:T
        fill!(S, false)
        lhsvalue = 0.  #x(L\S) + sum{d[j:l]*y[j] for j in S}
        empty = true
        for j in 1:l
            if x_val[j] > sumd[j,l]*y_val[j] + TOL
                S[j] = true
                empty = false
                lhsvalue += sumd[j,l]*y_val[j]
            else
                lhsvalue += x_val[j]
            end
        end
        if empty #fuerza que haya algo en S, si no la igualdad es trivial
            continue
        end
        if lhsvalue < sumd[1,l] - TOL
            lhs = sum(x[1:l])
            for j = (1:T)[S]
                lhs += sumd[j,l]*y[j] - x[j]
            end
            return lhs - sumd[1,l] #para retornar mayor que 0
        end
    end
    return
end

function readULS(path::String)
    f = open(path,"r")
    T = parse(Int64,readline(f))
    c = map(float,split(strip(readline(f))))
    h = map(float,split(strip(readline(f))))
    K = map(float,split(strip(readline(f))))
    d = map(float,split(strip(readline(f))))
    close(f)
    T, c, h, K, d
end

function solveULS(path::String; solver=CplexSolver(), valid::Bool = true)

    T, c, h, K, d = readULS(path)
    m = Model(solver = solver)
    @variable(m, y[1:T], Bin)
    @variable(m, x[i = 1:T] >= 0)
    @variable(m, s[1:T] >= 0)

    @objective(m, Min, sum(c[t]*x[t] + K[t]*y[t] + h[t]*s[t] for t in 1:T))
    @constraint(m, activation[t = 1:T], x[t] <= sum(d[t:T])*y[t])
    @constraint(m, balance[t = 1:T], (t>1?s[t-1]:0) + x[t] == s[t] + d[t])

    #precompute sum(d[j:l])
    sumd = zeros(Float64, T, T)
    for l = 1:T, j = 1:l
        sumd[j,l] = sum(d[j:l])
    end
    
    separationtime = 0.
    separated = 0
    called = 0
    function lSgenerator(cb)
        tt = time()
        called += 1
        y_val = getvalue(y)
        x_val = getvalue(x)
     
        expr = separate(T, sumd, y_val, x_val, y, x)
        if expr != nothing
            @usercut(cb, expr >= 0)
            separated += 1
        end
        separationtime += time()-tt
    end
    if valid
        addcutcallback(m, lSgenerator)
    end
    status = solve(m)
    println("Objective value: ", getobjectivevalue(m))
    println("Separation time: $separationtime seconds")
    println("Separated: $separated")
    status
end

In [3]:
run(`wget -O test.dat https://raw.githubusercontent.com/sbebo/julia-cuts/master/test.dat`)

--2017-10-17 19:59:48--  https://raw.githubusercontent.com/sbebo/julia-cuts/master/test.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1666 (1.6K) [text/plain]
Saving to: ‘test.dat’

     0K .                                                     100%  730M=0s

2017-10-17 19:59:49 (730 MB/s) - ‘test.dat’ saved [1666/1666]



In [ ]:
#solveULS("test.dat",solver=CplexSolver(CPX_PARAM_CUTSFACTOR=1,CPX_PARAM_SCRIND=0),valid=false)
solveULS("test.dat",solver=CplexSolver(CPX_PARAM_CUTSFACTOR=0,CPX_PARAM_SCRIND=1),valid=false)

## Absolute MMR SCP